In [3]:
import pandas as pd
import os
from bs4 import BeautifulSoup

Get data file and the list of roads that we need to extract data from.

In [4]:
cleaned_data_path = '..\data\processed\cleaned_data_final.csv'
cleaned_data = pd.read_csv(cleaned_data_path)
roads = cleaned_data['road'].unique().tolist()
roads

['N1',
 'N102',
 'N104',
 'N105',
 'N2',
 'N204',
 'N207',
 'R140',
 'R151',
 'R170',
 'R203',
 'R220',
 'R240',
 'R241',
 'R250',
 'R301',
 'R310',
 'R360',
 'Z1005',
 'Z1031',
 'Z1034',
 'Z1042',
 'Z1044',
 'Z1048',
 'Z1402',
 'Z2013']

In the code below we look through the RMMS folder to find the relevant .traffic.htm files with the data that we need for the roads found above. The tables in these .traffic.htm files are then extracted and added as a dataframe. All dataframes with road data are then concatenated to create one big dataframe.

In [5]:
traffic_data_path = '../data/raw/RMMS'
dfs = []

# Loop through each file in the directory
for filename in os.listdir(traffic_data_path):
    for road in roads:
        file_path = road + '.traffic.htm'
        if filename == file_path:
            file_path = os.path.join(traffic_data_path, filename)
            # Read the HTML file
            with open(file_path, 'r') as file:
                html_content = file.read()
            # Parse the HTML content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find the table(s) in the HTML content
            tables = soup.find_all('table')
            table = tables[4]

            # Extract data from the table and store it in a list of lists
            data = []
            for row in table.find_all('tr'):
                row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
                #print(row_data)
                data.append(row_data)

            columns = ['Link_No', 'Name', 'Start_LRP', 'Start_Offset', 'Start_Chainage', 'End_LRP',
                       'End_Offset', 'End_Chainage', 'Length_Km', 'Heavy_Truck', 'Medium_Truck',
                       'Small_Truck', 'Large_Bus', 'Medium_Bus', 'Micro_Bus', 'Utility', 'Car',
                       'Auto_Rickshaw', 'Motorcycle', 'Bicycle', 'Cycle_Rickshaw', 'Cart', 'Motorised_AADT',
                       'Non_Motorised_AADT', 'Total_AADT', 'AADT']
            # Create a DataFrame from the extracted data
            df = pd.DataFrame(data[3:], columns=columns)
            dfs.append(df)

traffic_dfs = pd.concat(dfs, ignore_index=True)
traffic_dfs


,Link_No,Name,Start_LRP,Start_Offset,Start_Chainage,End_LRP,End_Offset,End_Chainage,Length_Km,Heavy_Truck,...,Car,Auto_Rickshaw,Motorcycle,Bicycle,Cycle_Rickshaw,Cart,Motorised_AADT,Non_Motorised_AADT,Total_AADT,AADT
0,N1-1L,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1-1R,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1-2L,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1-3L,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Z1048-2,Nangalkot(int.with Z1415)-Chewara Bazar,LRP011,1209,12.292,LRPE,0,31.13,18.838,NS,...,NS,NS,NS,NS,NS,NS,NS,NS,NS,*NS
272,Z1402-1,Baburhat-Matlab Dakhin (int.with Z1445),LRPS,0,0,LRP010,3425,13.632,13.632,0.0,...,72.0,1364.0,31.0,20.0,40.0,0.0,1693.0,60.0,1753.0,1753.0
273,Z1402-2,Matlab Dakhin(int.with Z1445)-Narayanpur (int....,LRP010,3425,13.632,LRP019,1354,20.429,6.797,0.0,...,72.0,1364.0,31.0,20.0,40.0,0.0,1693.0,60.0,1753.0,1753.0
274,Z1402-3,Narayanpur (int.with Z1482)-Pennai (int.with N1),LRP019,1354,20.429,LRPE,0,43.849,23.420,NS,...,NS,NS,NS,NS,NS,NS,NS,NS,NS,*NS


In the following section we make the values float so we can add them together. Because L and R sections (N1-1L and N1-1R for example) end at the same location, we add them together to compute the total traffic for that section.

In [6]:
# Convert columns from index 9 onwards to float
traffic_dfs.iloc[:, 9:] = traffic_dfs.iloc[:, 9:].apply(pd.to_numeric, errors='coerce')

# Replace NaN values with 0
traffic_dfs.iloc[:, 9:] = traffic_dfs.iloc[:, 9:].fillna(0)

# Change all AADT values to float
traffic_dfs.iloc[:, 9:] = traffic_dfs.iloc[:, 9:].astype(float)

In [7]:
cleaned_traffic_dfs = []
columns = ['Link_No', 'Name', 'Start_LRP', 'Start_Offset', 'Start_Chainage', 'End_LRP',
                       'End_Offset', 'End_Chainage', 'Length_Km', 'Heavy_Truck', 'Medium_Truck',
                       'Small_Truck', 'Large_Bus', 'Medium_Bus', 'Micro_Bus', 'Utility', 'Car',
                       'Auto_Rickshaw', 'Motorcycle', 'Bicycle', 'Cycle_Rickshaw', 'Cart', 'Motorised_AADT',
                       'Non_Motorised_AADT', 'Total_AADT', 'AADT']
# Iterate through dataframe
for id, row in traffic_dfs.iterrows():
    # Check whether it is 1) not the end of the df and 2) whether the start and end chainages match with the previous row (indicating a row that needs merging)
    if (id != len(traffic_dfs) - 1) and (row['Start_Chainage'] == traffic_dfs.iloc[id+1]['Start_Chainage']) and (row['End_Chainage'] == traffic_dfs.iloc[id+1]['End_Chainage']):
        merged_row = []
        row1 = row
        row2 = traffic_dfs.iloc[id+1]
        # Merge rows
        for val1, val2 in zip(row1, row2):
            if val1 == val2:
                merged_row.append(val1)
            else:
                merged_row.append(val1 + val2)
        # Append to df
        merged_dataframe = pd.DataFrame([merged_row], columns=columns)
        cleaned_traffic_dfs.append(merged_dataframe)

    # If it is not a row that needs merging, just add it to the dataframe
    elif (id != 0) and (row['Start_Chainage'] != traffic_dfs.iloc[id-1]['Start_Chainage']) and (row['End_Chainage'] != traffic_dfs.iloc[id-1]['End_Chainage']):
        df = pd.DataFrame([row], columns=columns)
        cleaned_traffic_dfs.append(df)
            
cleaned_traffic_df = pd.concat(cleaned_traffic_dfs, ignore_index=True)

In [8]:
cleaned_traffic_df

,Link_No,Name,Start_LRP,Start_Offset,Start_Chainage,End_LRP,End_Offset,End_Chainage,Length_Km,Heavy_Truck,...,Car,Auto_Rickshaw,Motorcycle,Bicycle,Cycle_Rickshaw,Cart,Motorised_AADT,Non_Motorised_AADT,Total_AADT,AADT
0,N1-1LN1-1R,Jatrabari - Int.with Z1101 (Left) (Left)Jatrab...,LRPS,0,0,LRPS,822,0.822,0.822,1062.0,...,4459.0,5488.0,834.0,445.0,1977.0,0.0,38472.0,2422.0,40894.0,40894.0
1,N1-2LN1-2R,Int.with Z1101 - Signboard (Left) R111 (Left)I...,LRPS,822,0.822,LRPS,4175,4.175,3.353,1062.0,...,4459.0,5488.0,834.0,445.0,1977.0,0.0,38472.0,2422.0,40894.0,40894.0
2,N1-3LN1-3R,Signboard - Shimrail (Left)R110 (Left)Signboar...,LRPS,4175,4.175,LRPS,7181,7.181,3.006,176.0,...,3299.0,4423.0,2122.0,147.0,2338.0,0.0,31733.0,2485.0,34218.0,34218.0
3,N1-4LN1-4R,Shimrail - Katchpur (Left)N2 (Left)Shimrail ...,LRPS,7181,7.181,LRP009,260,8.763,1.582,327.0,...,3527.0,5847.0,2305.0,728.0,2608.0,0.0,31997.0,3336.0,35333.0,35333.0
4,N1-5LN1-5R,Katchpur - Madanpur (Left)N105 (Left)Katchpur ...,LRP009,260,8.763,LRP012,439,11.936,3.173,327.0,...,3684.0,6234.0,2297.0,728.0,2608.0,0.0,45132.0,3336.0,48468.0,48468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Z1048-2,Nangalkot(int.with Z1415)-Chewara Bazar,LRP011,1209,12.292,LRPE,0,31.13,18.838,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
233,Z1402-1,Baburhat-Matlab Dakhin (int.with Z1445),LRPS,0,0,LRP010,3425,13.632,13.632,0.0,...,72.0,1364.0,31.0,20.0,40.0,0.0,1693.0,60.0,1753.0,1753.0
234,Z1402-2,Matlab Dakhin(int.with Z1445)-Narayanpur (int....,LRP010,3425,13.632,LRP019,1354,20.429,6.797,0.0,...,72.0,1364.0,31.0,20.0,40.0,0.0,1693.0,60.0,1753.0,1753.0
235,Z1402-3,Narayanpur (int.with Z1482)-Pennai (int.with N1),LRP019,1354,20.429,LRPE,0,43.849,23.420,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
road_dics = {}

# Determine AADP per road
for id, row in cleaned_traffic_df.iterrows():
    for road in roads:
        if row['Link_No'].startswith(road + '-'):
            # Check whether it's the first entry in the dictionary
            if road_dics.get(road): 
                road_dics[road] = road_dics[road] + row['Heavy_Truck'] + row['Medium_Truck'] + row['Small_Truck']
            else:
                road_dics[road] = row['Heavy_Truck'] + row['Medium_Truck'] + row['Small_Truck']

In [10]:
road_dics

{'N1': 526693.0,
 'N102': 20061.0,
 'N104': 14994.0,
 'N105': 37030.0,
 'N2': 87964.0,
 'N204': 732.0,
 'N207': 3064.0,
 'R140': 29022.0,
 'R151': 2999.0,
 'R170': 3630.0,
 'R203': 1203.0,
 'R220': 210.0,
 'R240': 1899.0,
 'R241': 311.0,
 'R250': 3340.0,
 'R301': 25894.0,
 'R310': 6274.0,
 'R360': 10149.0,
 'Z1005': 324.0,
 'Z1031': 285.0,
 'Z1034': 515.0,
 'Z1042': 173.0,
 'Z1044': 208.0,
 'Z1048': 55.0,
 'Z1402': 286.0,
 'Z2013': 174.0}

In [15]:
road_weight = pd.DataFrame.from_dict(road_dics, orient='index', columns=['AADT'])
AADT_sum = sum(road_dics.values())
road_weight.reset_index(inplace=True)
road_weight = road_weight.rename(columns={'index': 'Road'})

# Calculate weight (i.e. percentage) by dividing by the sum of AADT
road_weight['Weight'] = (road_weight['AADT'] / AADT_sum)
value_count = cleaned_data['road'].loc[(cleaned_data['model_type'] == 'sourcesink')].value_counts()
road_weight_final = pd.DataFrame(columns=['Road', 'AADT', 'Weight'])
# Divide by number of sourcesinks to make sure roads with 2 sourcesinks don't produce double the Vehicles in the model
for i, row in road_weight.iterrows():
   
    road = row['Road']
    AADT = row['AADT']
    try:
        weight = row['Weight']/value_count.loc[row['Road']]
    except: # Roads with no sourcesinks don't get generated traffic
        weight = 0
    road_weight_row = pd.DataFrame([[road, AADT, weight]], columns=['Road', 'AADT', 'Weight'])
    road_weight_final = pd.concat([road_weight_final, road_weight_row], ignore_index=True)

    
    
road_weight_final

,Road,AADT,Weight
0,N1,526693.0,0.338714
1,N102,20061.0,0.000000
2,N104,14994.0,0.009643
3,N105,37030.0,0.023814
4,N2,87964.0,0.113139
5,N204,732.0,0.000941
6,N207,3064.0,0.000000
7,R140,29022.0,0.018664
8,R151,2999.0,0.003857
9,R170,3630.0,0.004669


In [16]:
# Save to csv
road_weight.to_csv('../data/processed/road_weights.csv', index=True)